In [ ]:
# 필요한 라이브러리 import
import time
import os
from Raspbot_Lib import Raspbot
from ipywidgets import interact
import ipywidgets as widgets
import sys

sys.path.append("/home/pi/software/oled_yahboom/")
from yahboom_oled import *

In [ ]:
# Raspbot 및 OLED 객체 생성
bot = Raspbot()
oled = Yahboom_OLED(debug=False)

In [ ]:
# ============================================
# 4방향 라인 추적 센서 관련 함수들
# ============================================


def read_line_tracker_sensor(bot_instance):
    """
    라인 추적 센서의 원시 데이터를 읽어옵니다.

    Args:
        bot_instance: Raspbot 객체

    Returns:
        int: 센서 원시 데이터 값, 실패 시 -1
    """
    if bot_instance is None:
        return -1

    try:
        track_data = bot_instance.read_data_array(0x0A, 1)
        track = int(track_data[0])
        return track
    except Exception as e:
        print(f"Error reading line tracker sensor: {e}")
        return -1


def parse_line_tracker_status(track_data):
    """
    라인 추적 센서의 원시 데이터를 4개 센서 상태로 파싱합니다.

    Args:
        track_data: 센서 원시 데이터 (8비트)

    Returns:
        tuple: (x2, x1, x3, x4) 각 센서의 상태 (0 또는 1)
    """
    if track_data < 0:
        return (0, 0, 0, 0)

    # 비트 마스킹을 통해 각 센서 상태 추출
    # 비트 위치: x1=비트3, x2=비트2, x3=비트1, x4=비트0
    x1 = (track_data >> 3) & 0x01
    x2 = (track_data >> 2) & 0x01
    x3 = (track_data >> 1) & 0x01
    x4 = track_data & 0x01

    return (x2, x1, x3, x4)


def display_line_tracker_status_on_oled(oled_instance, sensor_status):
    """
    OLED 화면에 라인 추적 센서 상태를 표시합니다.

    Args:
        oled_instance: Yahboom_OLED 객체
        sensor_status: (x2, x1, x3, x4) 튜플 형태의 센서 상태
    """
    if oled_instance is None:
        return

    x2, x1, x3, x4 = sensor_status
    oled_instance.clear()
    status_str = f"         {x2}  {x1}  {x3}  {x4}"
    oled_instance.add_line("Line Tracker Status:", 1)
    oled_instance.add_line("        x2 x1 x3 x4", 3)
    oled_instance.add_line(status_str, 4)
    oled_instance.refresh()


def cleanup_line_tracker_sensor():
    """
    라인 추적 센서 프로그램 종료 시 정리 작업을 수행합니다.
    """
    # OLED 기본 프로세스 복원
    os.system("python3 /home/pi/software/oled_yahboom/yahboom_oled.py &")
    print("Line tracking stopped by user.")


def main_line_tracker_monitoring(bot_instance, oled_instance, read_interval=0.1):
    """
    라인 추적 센서 모니터링 메인 루프

    Args:
        bot_instance: Raspbot 객체
        oled_instance: Yahboom_OLED 객체
        read_interval: 센서 읽기 간격 (초 단위, 기본값: 0.1초)
    """
    # 1. OLED 초기화
    oled_instance.init_oled_process()

    try:
        # 2. 메인 모니터링 루프
        while True:
            # 2-1. 라인 추적 센서 데이터 읽기
            track_data = read_line_tracker_sensor(bot_instance)

            # 2-2. 센서 상태 파싱
            sensor_status = parse_line_tracker_status(track_data)
            x2, x1, x3, x4 = sensor_status

            # 2-3. 센서 상태 출력
            print(f"Line Tracker Status: {x2}, {x1}, {x3}, {x4}")

            # 2-4. OLED에 센서 상태 표시
            display_line_tracker_status_on_oled(oled_instance, sensor_status)

            # 2-5. 읽기 간격 대기
            time.sleep(read_interval)

    except KeyboardInterrupt:
        # 3. 사용자 중단 시 정리 작업
        print("\nInterrupted by user.")
    finally:
        # 4. 종료 정리 작업
        cleanup_line_tracker_sensor()


# ============================================
# 메인 실행 함수
# ============================================
def main():
    """메인 실행 함수"""
    main_line_tracker_monitoring(bot, oled, read_interval=0.1)


if __name__ == "__main__":
    main()

Process 1547 has been terminated.
Line Tracker Status: 1, 1, 1, 1
Line Tracker Status: 1, 1, 1, 1
Line Tracker Status: 1, 1, 1, 1
Line Tracker Status: 1, 1, 1, 1
Line Tracker Status: 1, 1, 1, 1
Line Tracker Status: 1, 1, 1, 1
Line Tracker Status: 1, 1, 1, 1
Line Tracker Status: 1, 1, 1, 1
Line Tracker Status: 1, 1, 1, 1
Line Tracker Status: 1, 1, 1, 1
Line Tracker Status: 1, 1, 1, 1
Line Tracker Status: 1, 1, 1, 1
Line Tracker Status: 1, 1, 1, 1
Line Tracker Status: 1, 1, 1, 1
Line Tracker Status: 1, 1, 1, 1
Line Tracker Status: 1, 1, 1, 1
Line Tracker Status: 1, 1, 1, 1
Line Tracker Status: 1, 1, 1, 1


In [ ]:
# 프로그램 종료 시 정리 작업
# 1. 라인 추적 센서 종료
cleanup_line_tracker_sensor()

# 2. 객체 삭제 (다른 프로그램에서 Raspbot 라이브러리 사용 시 충돌 방지)
del bot

NameError: name 'bot' is not defined